In [3]:
import os

In [4]:
%pwd

'/Users/adityakumar/BankCD-Prediction/research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'/Users/adityakumar/BankCD-Prediction'

Write entity

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict #reading schema file
    

COnfiguartion Manger

In [8]:
from src.bankcdProject.constant import *
from src.bankcdProject.utils.common import read_yaml,create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

Compoenents

In [14]:
import os
from src.bankcdProject import logger
import pandas as pd

[2024-11-12 15:47:28,938: INFO: utils: NumExpr defaulting to 8 threads.]


In [15]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

Pipeline

In [16]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-11-12 15:47:32,303: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-12 15:47:32,304: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-12 15:47:32,307: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-12 15:47:32,308: INFO: common: created directory at: artifacts]
[2024-11-12 15:47:32,309: INFO: common: created directory at: artifacts/data_validation]


/var/folders/tf/nb2206b133550_hpm_b_xr_40000gn/T/ipykernel_26746/1524571817.py:10: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(self.config.unzip_data_dir)
